In [1]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

import warnings
warnings.filterwarnings('ignore')

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
# tf.set_random_seed(30)
tf.random.set_seed(30)

In [4]:
base_dir= 'kaggle/input/gesture-recognition/Project_data/'
model_save_dir= 'kaggle/working/models/'

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [5]:
train_doc = np.random.permutation(open(base_dir+'train.csv').readlines())
val_doc = np.random.permutation(open(base_dir+'train.csv').readlines())
batch_size = 30 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [6]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', '/'+source_path, '; batch size =', batch_size)
    img_idx = np.round(np.linspace(0, 29, num_frames_to_sample)).astype(int)
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(t)//batch_size
        for batch in range(num_batches): # we iterate over the number of batches
            
            batch_data = np.zeros((batch_size, len(img_idx), img_height, img_width, 3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    image_resized = imresize(image,(img_height, img_width)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = (image_resized[:,:,0])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (image_resized[:,:,1])/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (image_resized[:,:,2])/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        remaining_seq = len(t)%batch_size
        if remaining_seq != 0:
            for folder in range(remaining_seq):
                imgs = os.listdir(source_path+'/'+ t[folder + ((num_batches)*batch_size)].split(';')[0])
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + ((num_batches)*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    image_resized = imresize(image,(img_height, img_width)).astype(np.float32)
                    
                    batch_data[folder,idx,:,:,0] = image_resized[:,:,0]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = image_resized[:,:,1]/255 #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = image_resized[:,:,2]/255 #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (num_batches*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do  

In [7]:
curr_dt_time = datetime.datetime.now()

train_path = base_dir+'train'
val_path = base_dir+'val'

num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)

num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)

num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 663
# epochs = 10


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [8]:
num_classes = 5

In [9]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, SimpleRNN, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D, Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
from keras.layers.recurrent import LSTM , GRU

In [10]:
# Moving the code to train any model into a function for simplictiy
def train_model(model, batch_size, num_epochs):
    train_generator = generator(train_path, train_doc, batch_size)
    val_generator = generator(val_path, val_doc, batch_size)
    
    curr_dt_time = datetime.datetime.now()
    model_name = model_save_dir+'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
    if not os.path.exists(model_name):
        os.makedirs(model_name)

    filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

    LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=4) # write the REducelronplateau code here
    callbacks_list = [checkpoint, LR]

        
    if (num_train_sequences%batch_size) == 0:
        steps_per_epoch = int(num_train_sequences/batch_size)
    else:
        steps_per_epoch = (num_train_sequences//batch_size) + 1

    if (num_val_sequences%batch_size) == 0:
        validation_steps = int(num_val_sequences/batch_size)
    else:
        validation_steps = (num_val_sequences//batch_size) + 1
        
    history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)
    
    K.clear_session()

    return history

In [11]:
num_frames_to_sample = 30
img_height, img_width = 160, 160

In [12]:
#write your model here

model_1 = Sequential()

model_1.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(num_frames_to_sample, img_height, img_width, 3)))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(32, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(64, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(128, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Flatten())
model_1.add(Dense(128,activation='relu'))

model_1.add(Dense(64,activation='relu'))

model_1.add(Dense(num_classes, activation='softmax'))

2022-02-20 01:02:16.118988: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 01:02:16.147987: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 01:02:16.148734: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-02-20 01:02:16.150474: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Now that you have written the model, the next step is to compile the model. When you print the summary of the model, you'll see the total number of parameters you have to train.

In [13]:
optimiser = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation (Activation)      (None, 30, 160, 160, 16)  0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_1 (Activation)    (None, 15, 80, 80, 32)    0         
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 7, 40, 40, 64)     1

In [16]:
# # Experimenting for maximum batch_size when img_height, img_width = 160, 160
for batch_size in range(10, 50, 10):
    history = train_model(model_1, batch_size = batch_size, num_epochs = 1)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
67/67 [==============================] - 353s 5s/step - loss: 1.5638 - categorical_accuracy: 0.2493 - val_loss: 1.3192 - val_categorical_accuracy: 0.4400

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1900_11_43.534839/model-00001-1.56377-0.24925-1.31916-0.44000.h5
Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 20
34/34 [==============================] - 253s 8s/step - loss: 1.2554 - categorical_accuracy: 0.4588 - val_loss: 0.9956 - val_categorical_accuracy: 0.5600

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1900_11_43.534839/model-00001-1.25543-0.45882-0.99559-0.56000.h5
Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 30
23/23 [==============================] - 254s 11s/step - loss: 1.0512 - categorical_accuracy: 0.5333 - val_loss: 1.1763 - val_categorical_accuracy: 0.5750

Epoc

ResourceExhaustedError:  OOM when allocating tensor with shape[40,16,30,160,160] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv3d/Conv3D (defined at <ipython-input-10-91cd96f32b93>:32) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_1010]

Function call stack:
train_function


## Method 1: Using Conv3D

### Model 1 (Conv3D)
    batch_size = 30
    num_epochs = 10
    img_height, img_width = 160, 160

In [17]:
num_frames_to_sample = 30
img_height, img_width = 160, 160

In [19]:
model_1 = Sequential()

model_1.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(num_frames_to_sample, img_height, img_width, 3)))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(32, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(64, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Conv3D(128, (2, 2, 2), padding='same'))
model_1.add(Activation('relu'))
model_1.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_1.add(Flatten())
model_1.add(Dense(128,activation='relu'))

model_1.add(Dense(64,activation='relu'))

model_1.add(Dense(num_classes, activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_1.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])

model_1.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 160, 160, 16)  0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_9 (Activation)    (None, 15, 80, 80, 32)    0         
_________________________________________________________________
max_pooling3d_9 (MaxPooling3 (None, 7, 40, 40, 32)     0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 7, 40, 40, 64)    

In [20]:
history_1 = train_model(model_1, batch_size = 30, num_epochs = 10)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - 261s 12s/step - loss: 1.6176 - categorical_accuracy: 0.2145 - val_loss: 1.6631 - val_categorical_accuracy: 0.2250

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1906_56_53.421991/model-00001-1.61757-0.21449-1.66309-0.22500.h5
Epoch 2/10
23/23 [==============================] - 256s 12s/step - loss: 1.4685 - categorical_accuracy: 0.3435 - val_loss: 1.3946 - val_categorical_accuracy: 0.4333

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1906_56_53.421991/model-00002-1.46848-0.34348-1.39464-0.43333.h5
Epoch 3/10
23/23 [==============================] - 258s 12s/step - loss: 1.2490 - categorical_accuracy: 0.4681 - val_loss: 1.0850 - val_categorical_accuracy: 0.6417

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1906_56_53.421991/model-00003-1.24901-0.46812-1.08497-0.64167.h5
Epoch

### Model 2 (Conv3D)
    batch_size = 30
    num_epochs = 10
    img_height, img_width = 160, 160

In [21]:
num_frames_to_sample = 30
img_height, img_width = 160, 160

In [22]:
model_2 = Sequential()

model_2.add(Conv3D(16, (3,3,3), padding='same', input_shape=(num_frames_to_sample, img_height, img_width,3)))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(32, (3,3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(64, (3,3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Conv3D(128, (3,3,3), padding='same'))
model_2.add(Activation('relu'))
model_2.add(BatchNormalization())
model_2.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_2.add(Flatten())

model_2.add(Dense(128,activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.1))

model_2.add(Dense(128,activation='relu'))
model_2.add(BatchNormalization())
model_2.add(Dropout(0.1))

model_2.add(Dense(num_classes,activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_2.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 30, 160, 160, 16)  1312      
_________________________________________________________________
activation_12 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_12 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 15, 80, 80, 32)    13856     
_________________________________________________________________
activation_13 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 80, 80, 32)   

In [23]:
history_2 = train_model(model_2, batch_size = 30, num_epochs = 10)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 30
Epoch 1/10
23/23 [==============================] - 260s 12s/step - loss: 1.3513 - categorical_accuracy: 0.5072 - val_loss: 2.0041 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1908_15_45.231892/model-00001-1.35128-0.50725-2.00409-0.20000.h5
Epoch 2/10
23/23 [==============================] - 259s 12s/step - loss: 0.7465 - categorical_accuracy: 0.7435 - val_loss: 2.4798 - val_categorical_accuracy: 0.1833

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1908_15_45.231892/model-00002-0.74645-0.74348-2.47980-0.18333.h5
Epoch 3/10
23/23 [==============================] - 256s 12s/step - loss: 0.4605 - categorical_accuracy: 0.8478 - val_loss: 2.4439 - val_categorical_accuracy: 0.1667

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1908_15_45.231892/model-00003-0.46048-0.84783-2.44390-0.16667.h5
Epoch

### Model 3 (Conv3D)
    batch_size = 20
    num_epochs = 20
    img_height, img_width = 160, 160

In [24]:
num_frames_to_sample = 30
img_height, img_width = 160, 160

In [25]:
model_3 = Sequential()
model_3.add(Conv3D(16, (2,2,2), padding='same', input_shape=(num_frames_to_sample, img_height, img_width,3)))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Conv3D(32, (2,2,2), padding='same'))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Conv3D(64, (2,2,2), padding='same'))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Conv3D(128, (2,2,2), padding='same'))
model_3.add(Activation('relu'))
model_3.add(BatchNormalization())
model_3.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_3.add(Flatten())

model_3.add(Dense(256,activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.5))

model_3.add(Dense(256,activation='relu'))
model_3.add(BatchNormalization())
model_3.add(Dropout(0.5))

model_3.add(Dense(num_classes,activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_3.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_16 (Conv3D)           (None, 30, 160, 160, 16)  400       
_________________________________________________________________
activation_16 (Activation)   (None, 30, 160, 160, 16)  0         
_________________________________________________________________
batch_normalization_6 (Batch (None, 30, 160, 160, 16)  64        
_________________________________________________________________
max_pooling3d_16 (MaxPooling (None, 15, 80, 80, 16)    0         
_________________________________________________________________
conv3d_17 (Conv3D)           (None, 15, 80, 80, 32)    4128      
_________________________________________________________________
activation_17 (Activation)   (None, 15, 80, 80, 32)    0         
_________________________________________________________________
batch_normalization_7 (Batch (None, 15, 80, 80, 32)   

In [27]:
history_3 = train_model(model_3, batch_size = 20, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 20
Epoch 1/20
34/34 [==============================] - 255s 8s/step - loss: 1.6482 - categorical_accuracy: 0.4618 - val_loss: 5.7820 - val_categorical_accuracy: 0.1600

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1911_21_19.152452/model-00001-1.64823-0.46176-5.78204-0.16000.h5
Epoch 2/20
34/34 [==============================] - 253s 8s/step - loss: 1.1179 - categorical_accuracy: 0.6309 - val_loss: 9.4975 - val_categorical_accuracy: 0.1900

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1911_21_19.152452/model-00002-1.11789-0.63088-9.49751-0.19000.h5
Epoch 3/20
34/34 [==============================] - 257s 8s/step - loss: 0.9169 - categorical_accuracy: 0.6750 - val_loss: 13.3742 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1911_21_19.152452/model-00003-0.91693-0.67500-13.37416-0.23000.h5
Epoch 

### Model 4 (Conv3D)
    batch_size = 20
    num_epochs = 30
    img_height, img_width = 120, 120

In [28]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [30]:
model_4 = Sequential()

model_4.add(Conv3D(16, (3,3,3), padding='same', input_shape=(num_frames_to_sample, img_height, img_width,3)))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(Conv3D(16, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(32, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(Conv3D(32, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(64, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(Conv3D(64, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Conv3D(128, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(Conv3D(128, (3,3,3), padding='same'))
model_4.add(Activation('relu'))
model_4.add(BatchNormalization())

model_4.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_4.add(Flatten())
model_4.add(Dense(256,activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.5))

model_4.add(Dense(256,activation='relu'))
model_4.add(BatchNormalization())
model_4.add(Dropout(0.5))


model_4.add(Dense(num_classes,activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_4.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_4.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_28 (Conv3D)           (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_28 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_22 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
conv3d_29 (Conv3D)           (None, 30, 120, 120, 16)  6928      
_________________________________________________________________
activation_29 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_24 (MaxPooling (None, 15, 60, 60, 16)   

In [32]:
history_4 = train_model(model_4, batch_size = 20, num_epochs = 30)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 214s 6s/step - loss: 2.0917 - categorical_accuracy: 0.3529 - val_loss: 1.6425 - val_categorical_accuracy: 0.3200

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1912_45_30.473179/model-00001-2.09175-0.35294-1.64245-0.32000.h5
Epoch 2/30
34/34 [==============================] - 220s 7s/step - loss: 1.5472 - categorical_accuracy: 0.4588 - val_loss: 3.9207 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1912_45_30.473179/model-00002-1.54723-0.45882-3.92065-0.22000.h5
Epoch 3/30
34/34 [==============================] - 210s 6s/step - loss: 1.4660 - categorical_accuracy: 0.5044 - val_loss: 9.5107 - val_categorical_accuracy: 0.1600

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1912_45_30.473179/model-00003-1.46604-0.50441-9.51074-0.16000.h5
Epoch 4/

### Model 5 (Conv3D)
    batch_size = 20
    num_epochs = 30
    img_height, img_width = 120, 120

In [33]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [36]:
model_5 = Sequential()
model_5.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(num_frames_to_sample, img_height, img_width,3)))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(32, (2, 2, 2), padding='same'))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(64, (2, 2, 2), padding='same'))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Conv3D(128, (2, 2, 2), padding='same'))
model_5.add(Activation('relu'))
model_5.add(BatchNormalization())
model_5.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_5.add(Flatten())
model_5.add(Dense(128,activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.25))

model_5.add(Dense(128,activation='relu'))
model_5.add(BatchNormalization())
model_5.add(Dropout(0.25))

model_5.add(Dense(num_classes,activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_5.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_5.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_44 (Conv3D)           (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_44 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_44 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_36 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_45 (Conv3D)           (None, 15, 60, 60, 32)    4128      
_________________________________________________________________
activation_45 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_45 (Batc (None, 15, 60, 60, 32)   

In [37]:
history_5 = train_model(model_5, batch_size = 20, num_epochs = 30)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 20
Epoch 1/30
34/34 [==============================] - 210s 6s/step - loss: 1.4392 - categorical_accuracy: 0.4750 - val_loss: 3.1507 - val_categorical_accuracy: 0.2400

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1914_16_54.364892/model-00001-1.43916-0.47500-3.15071-0.24000.h5
Epoch 2/30
34/34 [==============================] - 204s 6s/step - loss: 0.8944 - categorical_accuracy: 0.6132 - val_loss: 3.5748 - val_categorical_accuracy: 0.2100

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1914_16_54.364892/model-00002-0.89437-0.6132-3.57484-0.21000.h5
Epoch 3/30
34/34 [==============================] - 209s 6s/step - loss: 0.6422 - categorical_accuracy: 0.6779 - val_loss: 4.2236 - val_categorical_accuracy: 0.1900

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1914_16_54.364892/model-00003-0.64215-0.6779-4.22356-0.19000.h5
Epoch 4/30

### Model 6 (Conv3D)
    batch_size = 30
    num_epochs = 20
    img_height, img_width = 120, 120

In [41]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [43]:
model_6 = Sequential()
model_6.add(Conv3D(16, (3, 3, 3), padding='same', input_shape=(num_frames_to_sample, img_height,img_width,3)))
model_6.add(Activation('relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(32, (3, 3, 3), padding='same'))
model_6.add(Activation('relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(64, (2, 2, 2), padding='same'))
model_6.add(Activation('relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Conv3D(128, (2, 2, 2), padding='same'))
model_6.add(Activation('relu'))
model_6.add(BatchNormalization())
model_6.add(MaxPooling3D(pool_size=(2, 2, 2)))

model_6.add(Flatten())
model_6.add(Dense(64,activation='relu'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.3))

model_6.add(Dense(64,activation='relu'))
model_6.add(BatchNormalization())
model_6.add(Dropout(0.3))

model_6.add(Dense(num_classes,activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_6.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_6.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_60 (Conv3D)           (None, 30, 120, 120, 16)  1312      
_________________________________________________________________
activation_60 (Activation)   (None, 30, 120, 120, 16)  0         
_________________________________________________________________
batch_normalization_68 (Batc (None, 30, 120, 120, 16)  64        
_________________________________________________________________
max_pooling3d_52 (MaxPooling (None, 15, 60, 60, 16)    0         
_________________________________________________________________
conv3d_61 (Conv3D)           (None, 15, 60, 60, 32)    13856     
_________________________________________________________________
activation_61 (Activation)   (None, 15, 60, 60, 32)    0         
_________________________________________________________________
batch_normalization_69 (Batc (None, 15, 60, 60, 32)  

In [44]:
history_6 = train_model(model_6, batch_size = 20, num_epochs = 30)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 20
Epoch 1/25
34/34 [==============================] - 215s 6s/step - loss: 1.6071 - categorical_accuracy: 0.3971 - val_loss: 2.4000 - val_categorical_accuracy: 0.2000

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-1916_56_53.421991/model-00001-1.60714-0.39706-2.39998-0.20000.h5
Epoch 2/25
34/34 [==============================] - 209s 6s/step - loss: 1.0896 - categorical_accuracy: 0.5662 - val_loss: 4.3015 - val_categorical_accuracy: 0.1800

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-1916_56_53.421991/model-00002-1.08961-0.56618-4.30147-0.18000.h5
Epoch 3/25
34/34 [==============================] - 208s 6s/step - loss: 0.8052 - categorical_accuracy: 0.6971 - val_loss: 4.5880 - val_categorical_accuracy: 0.1800

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-1916_56_53.421991/model-00003-0.80520-0.69706-4.58797-0.18000.h5
Epoch 4/

## Method 2: using CNN + RNN

### Model 7 (CNN + SimpleRNN)
    batch_size = 10
    num_epochs = 10
    img_height, img_width = 120, 120

In [45]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [46]:
model_7 = Sequential()

model_7.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape = (num_frames_to_sample, img_height, img_width, 3)))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(MaxPooling2D((2, 2))))

model_7.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(MaxPooling2D((2, 2))))

model_7.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(MaxPooling2D((2, 2))))

model_7.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_7.add(TimeDistributed(BatchNormalization()))
model_7.add(TimeDistributed(MaxPooling2D((2, 2))))

model_7.add(TimeDistributed(Flatten()))


model_7.add(SimpleRNN(256))

model_7.add(Dense(256,activation='relu'))
model_7.add(Dropout(0.3))

model_7.add(Dense(num_classes, activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_7.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_7.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_1 (TimeDist (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_2 (TimeDist (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_4 (TimeDist (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_5 (TimeDist (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 30, 30, 30, 64)  

In [48]:
history_7 = train_model(model_7, batch_size = 10, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 308s 5s/step - loss: 1.4132 - categorical_accuracy: 0.4224 - val_loss: 4.5476 - val_categorical_accuracy: 0.2100

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-2001_02_11.519018/model-00001-1.41318-0.42239-4.54758-0.21000.h5
Epoch 2/20
67/67 [==============================] - 249s 4s/step - loss: 1.0478 - categorical_accuracy: 0.5716 - val_loss: 4.8890 - val_categorical_accuracy: 0.2200

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-2001_02_11.519018/model-00002-1.04784-0.57164-4.88897-0.22000.h5
Epoch 3/20
67/67 [==============================] - 210s 3s/step - loss: 1.0298 - categorical_accuracy: 0.5925 - val_loss: 3.0285 - val_categorical_accuracy: 0.3200

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-2001_02_11.519018/model-00003-1.02983-0.59254-3.02852-0.32000.h5
Epoch 4/

### Model 8 (CNN + LSTM)
    batch_size = 10
    num_epochs = 10
    img_height, img_width = 120, 120

In [49]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [50]:
model_8 = Sequential()

model_8.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'), input_shape = (num_frames_to_sample, img_height, img_width, 3)))
model_8.add(TimeDistributed(BatchNormalization()))
model_8.add(TimeDistributed(MaxPooling2D((2, 2))))

model_8.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model_8.add(TimeDistributed(BatchNormalization()))
model_8.add(TimeDistributed(MaxPooling2D((2, 2))))

model_8.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model_8.add(TimeDistributed(BatchNormalization()))
model_8.add(TimeDistributed(MaxPooling2D((2, 2))))

model_8.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model_8.add(TimeDistributed(BatchNormalization()))
model_8.add(TimeDistributed(MaxPooling2D((2, 2))))

model_8.add(TimeDistributed(Conv2D(256, (3, 3) , padding='same', activation='relu')))
model_8.add(TimeDistributed(BatchNormalization()))
model_8.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model_8.add(TimeDistributed(Flatten()))

model_8.add(LSTM(128))

model_8.add(Dense(128,activation='relu'))
model_8.add(Dropout(0.3))

model_8.add(Dense(num_classes, activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_8.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_8.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_13 (TimeDis (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_14 (TimeDis (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_15 (TimeDis (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_16 (TimeDis (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_17 (TimeDis (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_18 (TimeDis (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 30, 30, 30, 64)  

In [51]:
history_8 = train_model(model_8, batch_size = 10, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 224s 3s/step - loss: 1.4838 - categorical_accuracy: 0.3597 - val_loss: 1.7960 - val_categorical_accuracy: 0.1800

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-2003_02_53.824697/model-00001-1.48384-0.35970-1.79605-0.18000.h5
Epoch 2/20
67/67 [==============================] - 233s 4s/step - loss: 1.2171 - categorical_accuracy: 0.4955 - val_loss: 1.8753 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-2003_02_53.824697/model-00002-1.21712-0.49552-1.87529-0.25000.h5
Epoch 3/20
67/67 [==============================] - 217s 3s/step - loss: 0.9932 - categorical_accuracy: 0.6045 - val_loss: 2.1078 - val_categorical_accuracy: 0.2300

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-2003_02_53.824697/model-00003-0.99317-0.60448-2.10781-0.23000.h5
Epoch 4/

### Model-9 (CNN + LSTM) with Transfer learning

We have used the MobileNet pre-trained model as it has the 'Top-1' and 'Top-5' accuracy is almost similar to that of VGG-16 but with much lesser parameters just 4.3M as compared to VGG-16 which has 138.4M parameters. MobileNet also has a faster inference speed. (Source: https://keras.io/api/applications/)

In [52]:
from tensorflow.keras.applications import mobilenet
mobilenet_model = mobilenet.MobileNet(weights='imagenet', include_top=False)

17235968/17225924 [==============================] - 2s 0us/step


In [54]:
model_9 = Sequential()
model_9.add(TimeDistributed(mobilenet_model,input_shape = (num_frames_to_sample, img_height, img_width, 3)))

for layer in model_9.layers:
    layer.trainable = False

model_9.add(TimeDistributed(BatchNormalization()))
model_9.add(TimeDistributed(MaxPooling2D((2, 2))))
model_9.add(TimeDistributed(Flatten()))

model_9.add(LSTM(128))
model_9.add(Dropout(0.25))

model_9.add(Dense(128,activation='relu'))
model_9.add(Dropout(0.25))

model_9.add(Dense(num_classes, activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_9.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_9.summary()

Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_33 (TimeDis (None, 30, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_34 (TimeDis (None, 30, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_35 (TimeDis (None, 30, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_36 (TimeDis (None, 30, 1024)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dropout_26 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_48 (Dense)             (None, 128)             

In [63]:
history_9= train_model(model_9, batch_size = 10, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 220s 3s/step - loss: 1.4951 - categorical_accuracy: 0.3478 - val_loss: 1.2630 - val_categorical_accuracy: 0.4500

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-2010_48_02.580195/model-00001-1.49509-0.34776-1.26295-0.45000.h5
Epoch 2/20
67/67 [==============================] - 214s 3s/step - loss: 0.9218 - categorical_accuracy: 0.6791 - val_loss: 0.8342 - val_categorical_accuracy: 0.6300

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-2010_48_02.580195/model-00002-0.92179-0.67910-0.83417-0.63000.h5
Epoch 3/20
67/67 [==============================] - 213s 3s/step - loss: 0.6202 - categorical_accuracy: 0.7851 - val_loss: 0.6480 - val_categorical_accuracy: 0.7500

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-2010_48_02.580195/model-00003-0.62016-0.78507-0.64799-0.75000.h5
Epoch 4/

### Model-10 (CNN + LSTM) with Transfer learning and Training CNN layers

In [64]:
from tensorflow.keras.applications import mobilenet
mobilenet_model = mobilenet.MobileNet(weights='imagenet', include_top=False)

In [65]:
model_10 = Sequential()
model_10.add(TimeDistributed(mobilenet_model,input_shape = (num_frames_to_sample, img_height, img_width, 3)))

for layer in model_10.layers:
    layer.trainable = True

model_10.add(TimeDistributed(BatchNormalization()))
model_10.add(TimeDistributed(MaxPooling2D((2, 2))))
model_10.add(TimeDistributed(Flatten()))

model_10.add(LSTM(128))
model_10.add(Dropout(0.25))

model_10.add(Dense(128,activation='relu'))
model_10.add(Dropout(0.25))

model_10.add(Dense(num_classes, activation='softmax'))

optimiser = tf.keras.optimizers.Adam()
model_10.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_10.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_41 (TimeDis (None, 30, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_42 (TimeDis (None, 30, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_43 (TimeDis (None, 30, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_44 (TimeDis (None, 30, 1024)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 128)               590336    
_________________________________________________________________
dropout_30 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_52 (Dense)             (None, 128)             

In [62]:
history_10 = train_model(model_10, batch_size = 10, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 226s 3s/step - loss: 1.3385 - categorical_accuracy: 0.4343 - val_loss: 0.9031 - val_categorical_accuracy: 0.6700

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-2014_37_40.081350/model-00001-1.33846-0.43433-0.90307-0.67000.h5
Epoch 2/20
67/67 [==============================] - 222s 3s/step - loss: 0.8510 - categorical_accuracy: 0.6791 - val_loss: 0.8744 - val_categorical_accuracy: 0.6600

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-2014_37_40.081350/model-00002-0.85097-0.67910-0.87444-0.66000.h5
Epoch 3/20
67/67 [==============================] - 221s 3s/step - loss: 0.7106 - categorical_accuracy: 0.7716 - val_loss: 0.7269 - val_categorical_accuracy: 0.7000

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-2014_37_40.081350/model-00003-0.71057-0.77164-0.72694-0.70000.h5
Epoch 4/

### Model-11 (CNN + GRU) with Transfer learning and Training CNN layers

In [66]:
num_frames_to_sample = 30
img_height, img_width = 120, 120

In [67]:
from tensorflow.keras.applications import mobilenet
mobilenet_model = mobilenet.MobileNet(weights='imagenet', include_top=False)

In [72]:
model_11 = Sequential()
model_11.add(TimeDistributed(mobilenet_model, input_shape = (num_frames_to_sample, img_height, img_width, 3)))

for layer in model_11.layers:
    layer.trainable = True
    
model_11.add(TimeDistributed(BatchNormalization()))
model_11.add(TimeDistributed(MaxPooling2D((2, 2))))
model_11.add(TimeDistributed(Flatten()))

model_11.add(GRU(128))
model_11.add(Dropout(0.25))

model_11.add(Dense(128,activation='relu'))
model_11.add(Dropout(0.25))

model_11.add(Dense(num_classes, activation='softmax'))


optimiser = tf.keras.optimizers.Adam()
model_11.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_11.summary()

Model: "sequential_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_57 (TimeDis (None, 30, 3, 3, 1024)    3228864   
_________________________________________________________________
time_distributed_58 (TimeDis (None, 30, 3, 3, 1024)    4096      
_________________________________________________________________
time_distributed_59 (TimeDis (None, 30, 1, 1, 1024)    0         
_________________________________________________________________
time_distributed_60 (TimeDis (None, 30, 1024)          0         
_________________________________________________________________
gru_3 (GRU)                  (None, 128)               442752    
_________________________________________________________________
dropout_38 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_60 (Dense)             (None, 128)             

In [73]:
history_11= train_model(model_11, batch_size = 10, num_epochs = 20)

Source path =  /kaggle/input/gesture-recognition/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - 227s 3s/step - loss: 1.2113 - categorical_accuracy: 0.5045 - val_loss: 0.7186 - val_categorical_accuracy: 0.7400

Epoch 00001: saving model to /kaggle/working/models/model_init_2022-02-2015_52_27.164407/model-00001-1.21134-0.50448-0.71856-0.74000.h5
Epoch 2/20
67/67 [==============================] - 220s 3s/step - loss: 0.5852 - categorical_accuracy: 0.7836 - val_loss: 0.6624 - val_categorical_accuracy: 0.7800

Epoch 00002: saving model to /kaggle/working/models/model_init_2022-02-2015_52_27.164407/model-00002-0.58517-0.78358-0.66243-0.78000.h5
Epoch 3/20
67/67 [==============================] - 218s 3s/step - loss: 0.3766 - categorical_accuracy: 0.8776 - val_loss: 0.3286 - val_categorical_accuracy: 0.8300

Epoch 00003: saving model to /kaggle/working/models/model_init_2022-02-2015_52_27.164407/model-00003-0.37662-0.87761-0.32865-0.83000.h5
Epoch 4/

### Final Observation

The best model performance that we achieved was using Pre-trained MobileNet CNN model and GRU network. In which we also trained the weights of the CNN model. (Model 11)

`For Model 11:`
   - Training Accuracy was 100% and Validation Accuracy was 99%
   
The weights for the same model can be found at "model_5/model_init_2022-02-2015_52_27.164407/model-00016-0.06678-0.99403-0.04007-0.99000.h5"